# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home


In [3]:
file_path_list[0:5]

['/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home/event_data/2018-11-15-events.csv',
 '/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home/event_data/2018-11-22-events.csv',
 '/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home/event_data/2018-11-09-events.csv',
 '/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home/event_data/2018-11-18-events.csv',
 '/Users/zhou/Desktop/Udacity_study/数据工程师项目文件/Cassandra项目/home/event_data/2018-11-04-events.csv']

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [11]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
             
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [12]:

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [13]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [14]:
create_keyspace="""CREATE KEYSPACE IF NOT EXISTS udacity  WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
session.execute(create_keyspace)

#### Set Keyspace

In [15]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [14]:
df=pd.read_csv('./event_datafile_new.csv')

## #TABLE1:artist_info
description:

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

### cell 1:
- Since we want to SELECT out `artist`, `song title`, and `song's length` based on `sessionId `and `itemInSession`, we need to SET **parition key** as `(sessionId, itemInSession)`. 
- We name this table `artist_info`, to inform user this table explain some specific info about artist.

In [15]:
drop = """DROP TABLE IF EXISTS artist_info"""
query = """CREATE TABLE IF NOT EXISTS artist_info (
                    session_id INT,
                    item_in_session INT,
                    artist VARCHAR,
                    song_title VARCHAR,
                    length FLOAT,
                    PRIMARY KEY((session_id, item_in_session))
        )"""

try:
    session.execute(drop) 
    session.execute(query)       
except Exception as e:
    print(e)

## #TABLE1:artist_info
### cell 2:
- Do a `INSERT` to fill in the table

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = """INSERT INTO artist_info (session_id, item_in_session, artist, song_title, length)"""
        query = query + "Values (" + ','.join(['%s']*5) + ')'
        try:
            session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

## #TABLE1:artist_info
### cell 3:
- Do a `SELECT` to verify that the data have been inserted into each table

In [17]:
query = """SELECT artist, song_title, length FROM artist_info WHERE item_in_session = %s and session_id = %s"""

try:
    rows = session.execute(query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## #TABLE2:user_listening_history
description:

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### cell 1:
- Since we want to SELECT out `artist`, `song title`, `user` based on `userId `and `itemInSession`, we need to SET **partition key** as `(userId, itemInSession)`. 
- We also want what outcomes sorted by `itemInSession`, so we make it **clustering columns**. Adding `itemInSession` provdes enough identification for every single clue, so we don't need any further clustering columns
- We named this table `user_listening_history` as it uses some user information to find listening histories.

In [18]:
drop = """DROP TABLE IF EXISTS user_listening_history"""
query = """CREATE TABLE IF NOT EXISTS user_listening_history (
                    user_id INT,
                    session_id INT,
                    artist VARCHAR,
                    item_in_session INT,
                    song_title VARCHAR, 
                    user VARCHAR,
                    PRIMARY KEY((user_id, session_id), item_in_session)
        )"""
try:
    session.execute(drop) 
    session.execute(query)       
except Exception as e:
    print(e)


## #TABLE2:user_listening_history
### cell 2:
- Do a `INSERT` to fill in the table

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = """INSERT INTO user_listening_history (user_id, session_id, artist, item_in_session, song_title, user)"""
        query2 = query2 + "Values (" + ','.join(['%s']*6) + ')'
        try:
            session.execute(query2, (int(line[10]), int(line[8]), line[0], int(line[3]), line[9], line[1]+' '+line[4]))
        except Exception as e:
            print(e)

## #TABLE2:user_listening_history
### cell 3:
- Do a `SELECT` to verify that the data have been inserted into each table

In [20]:
query2 = """SELECT artist, song_title, user FROM user_listening_history WHERE user_id = %s and session_id = %s """

try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user='Sylvie Cruz')
Row(artist='Three Drives', song_title='Greece 2000', user='Sylvie Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', user='Sylvie Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


## #TABLE3:song_audience
description:

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### cell 1:
- Since we want to SELECT out `song title`, `user` based on `song title`, we need to SET **partition key** as `song title`.
- We also want all users are listed in the output, so we make `user` **clustering columns**. 
- We named this table `song_audience` as it uses song title to retrieve it's audience users.

In [21]:
drop = """DROP TABLE IF EXISTS song_audience"""
query3 = """CREATE TABLE IF NOT EXISTS song_audience (
                    song_title VARCHAR,
                    user VARCHAR,
                    PRIMARY KEY(song_title, user)
        )"""

try:
    session.execute(drop) 
    session.execute(query3)       
except Exception as e:
    print(e)
              

## #TABLE3:song_audience
### cell 2:
- Do a `INSERT` to fill in the table

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query3 = """INSERT INTO song_audience ( song_title, user)"""
        query3 = query3 + "Values (" + ','.join(['%s']*2) + ')'
        try:
            session.execute(query3, ( line[9], line[1]+' '+line[4]))
        except Exception as e:
            print(e)
     

## #TABLE3:song_audience
### cell 3:
- Do a `SELECT` to verify that the data have been inserted into each table

In [23]:
query3 = """SELECT user FROM song_audience WHERE song_title=%s"""

try:
    rows = session.execute(query3, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)


Row(user='Jacqueline Lynch')
Row(user='Sara Johnson')
Row(user='Tegan Levine')


### Drop the tables before closing out the sessions

In [24]:
drop = "DROP TABLE IF EXISTS "
try:
    for table in ["artist_info", "user_listening_history", "song_audience"]:
        session.execute(drop+str(i))
except Exception as e:
    print(e)

name 'i' is not defined


### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()